```{contents}
```
## Mocking 

**Mocking** in LangGraph refers to the systematic replacement of real components (LLMs, tools, APIs, databases, human inputs) with **controlled, deterministic stand-ins** for the purpose of **testing, debugging, simulation, and safe development** of LLM workflows.

Mocking allows you to test **graph logic** independently from **external dependencies**.

---

### **1. Why Mocking Is Essential in LangGraph**

LangGraph systems are:

* Stateful
* Multi-step
* Cyclic
* Tool-driven
* Often nondeterministic

Therefore, reliable development requires **isolating the graph’s control logic** from:

* LLM randomness
* Network instability
* Tool failures
* Human interaction

---

### **2. What Can Be Mocked**

| Component         | Why Mock                |
| ----------------- | ----------------------- |
| LLM calls         | Deterministic behavior  |
| Tools             | Avoid real side effects |
| APIs              | No external dependency  |
| Databases         | Faster tests            |
| Human input       | Automated testing       |
| Time / randomness | Reproducibility         |

---

### **3. Mocking Levels**

| Level       | Scope               |
| ----------- | ------------------- |
| Unit        | Individual node     |
| Integration | Subgraph            |
| System      | Full graph          |
| Simulation  | End-to-end scenario |

---

### **4. Basic Node Mocking Example**

```python
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    question: str
    answer: str

# Real node
def real_llm_node(state):
    return {"answer": call_real_llm(state["question"])}

# Mock node
def mock_llm_node(state):
    return {"answer": "mocked answer"}

builder = StateGraph(State)
builder.add_node("llm", mock_llm_node)   # swap real with mock
builder.set_entry_point("llm")
builder.add_edge("llm", END)

graph = builder.compile()
```

---

### **5. Mocking External Tools**

```python
def real_search(query): ...
def mock_search(query): return "static result"

def tool_node(state):
    return {"data": mock_search(state["query"])}
```

---

### **6. Mocking Human-in-the-Loop**

```python
def mock_human_node(state):
    return {"approved": True}
```

Used for automated regression testing of approval workflows.

---

### **7. Deterministic Multi-Step Testing**

```python
def mock_router(state):
    if state["step"] < 2:
        return "loop"
    return END
```

This allows deterministic testing of **cyclic graphs**.

---

### **8. Integration Testing Pattern**

```python
def test_graph():
    result = graph.invoke({"question": "test"})
    assert result["answer"] == "mocked answer"
```

---

### **9. Production-Grade Mocking Architecture**

| Environment | Configuration   |
| ----------- | --------------- |
| Local dev   | Full mocks      |
| CI pipeline | Full mocks      |
| Staging     | Partial mocks   |
| Production  | Real components |

---

### **10. Advanced Techniques**

| Technique            | Purpose                  |
| -------------------- | ------------------------ |
| Snapshot testing     | Verify state evolution   |
| Golden tests         | Compare expected outputs |
| Fault injection      | Test failure recovery    |
| Time travel          | Replay from checkpoints  |
| Property-based tests | Validate invariants      |

---

### **11. Mocking vs Simulation**

| Mocking             | Simulation          |
| ------------------- | ------------------- |
| Replace component   | Emulate environment |
| Deterministic       | Probabilistic       |
| Testing correctness | Testing behavior    |

---

### **12. Why LangGraph Needs Mocking More Than Normal Apps**

LangGraph systems are **control systems**.
Their correctness depends more on **flow logic** than raw outputs.

Mocking allows engineers to prove:

> **Given this state, the graph always makes the correct decision.**


### Demonstration

In [1]:
# Complete one-cell demonstration of mocking in LangGraph

from typing import TypedDict
from langgraph.graph import StateGraph, END

# ---------------------------
# 1. Define Shared State
# ---------------------------

class State(TypedDict):
    question: str
    step: int
    approved: bool
    answer: str

# ---------------------------
# 2. Real vs Mock Components
# ---------------------------

def real_llm(state):
    raise RuntimeError("Real LLM should not be called during tests")

def mock_llm(state):
    return {"answer": f"MOCK: answer to '{state['question']}'"}

def mock_tool(state):
    return {"step": state["step"] + 1}

def mock_human(state):
    return {"approved": True}

# ---------------------------
# 3. Router for Cyclic Flow
# ---------------------------

def router(state):
    if state["step"] < 2:
        return "loop"
    return "final"

# ---------------------------
# 4. Build Graph
# ---------------------------

builder = StateGraph(State)

builder.add_node("llm", mock_llm)
builder.add_node("tool", mock_tool)
builder.add_node("human", mock_human)

builder.set_entry_point("llm")
builder.add_edge("llm", "tool")

builder.add_conditional_edges(
    "tool",
    router,
    {"loop": "llm", "final": "human"}
)

builder.add_edge("human", END)

graph = builder.compile()

# ---------------------------
# 5. Deterministic Test Run
# ---------------------------

result = graph.invoke({
    "question": "What is LangGraph?",
    "step": 0,
    "approved": False,
    "answer": ""
})

# ---------------------------
# 6. Assertion (Mock Verification)
# ---------------------------

assert result["answer"].startswith("MOCK")
assert result["approved"] is True
assert result["step"] == 2

print("Test passed. Final state:\n", result)


Test passed. Final state:
 {'question': 'What is LangGraph?', 'step': 2, 'approved': True, 'answer': "MOCK: answer to 'What is LangGraph?'"}
